In [16]:
import os
import re
from collections import defaultdict
from typing import Dict


def find_function_usage(directory: str) -> Dict[str, Dict[str, int]]:
    """
    Find all occurrences of functions in .ts and .svelte files within a directory.
    Returns a dictionary mapping function names to files and their usage count.
    """

    # Updated regex to handle different function definitions and exports
    function_pattern = re.compile(r'''
        (?:export\s+)?                           # Optional export
        (?:function|const|let|var|async\s+function)  # Keywords for function definition
        \s+([a-zA-Z_]\w*)                         # Function name
        \s*(?:\(|=\s*(?:\(|\[|\{|async|\(\)\s*=>|\s*=>))  # Handle arrow functions and async
    ''', re.VERBOSE)

    # To store all functions found in the project
    function_names = set()

    # File extensions to scan
    extensions = ('.ts', '.svelte')

    # First pass: Collect all function names
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(extensions):
                filepath = os.path.join(root, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()
                        matches = function_pattern.findall(content)
                        function_names.update(matches)
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

    # Second pass: Count occurrences of each function
    usage_by_function = defaultdict(lambda: defaultdict(int))

    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(extensions):
                filepath = os.path.join(root, filename)
                relative_path = os.path.relpath(filepath, directory)

                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()

                        for function_name in function_names:
                            usage_pattern = re.compile(r'\b' + re.escape(function_name) + r'\b')
                            matches = usage_pattern.findall(content)

                            if matches:
                                usage_by_function[function_name][relative_path] += len(matches)
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

    return usage_by_function

In [17]:
# Get project directory from user (default to current directory)
directory = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"
print(f"Scanning {directory} for function usage...")
usage = find_function_usage(directory)
# Flatten the data into a list of rows for the CSV
rows = []
for func, files in usage.items():
    for file, count in files.items():
        rows.append([func, file, count])

# Write the rows to a CSV file
with open('function_file_counts.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Function', 'File Path', 'Count'])  # Write header
    writer.writerows(rows)  # Write data

Scanning C:\Users\harold.noble\Desktop\RIC\app\frontend for function usage...
